# SNLP 2021 Final Project

Name 1: Lukas Wilde<br/> 
Student id 1: 2564597<br/>
Email 1: s8luwild@stud.uni-saarland.de<br/>


Name 2: Dennis Heß<br/>
Student id 2: 2574005<br/>
Email 2: s8dehess@stud.uni-saarland.de<br/> 

In [59]:
import sentencepiece as sp
from nltk import sent_tokenize
import os
import random

In [60]:
random.seed(42)

PATH = "./data"

for name in ['alice_in_wonderland.txt', 'bengali_corpus.txt']:
    file = os.path.join(PATH, name)
    with open(file, 'r') as f:
        x = f.read()
        total_symbols = len(x)
        
        sentences = sent_tokenize(x)
        sentences = list(map(lambda x: " ".join(x.split()), sentences))
        
        num_sentences = len(sentences)
        
        train_set = sentences
        
        test_set = []
        test_symbols = 0
        
        while test_symbols < 0.2 * total_symbols:
            idx = random.randint(0, len(train_set)-1)
            test_sentence = train_set.pop(idx)
            test_symbols += len(test_sentence)
            test_set.append(test_sentence)
                
        print("train-test-split: %f %f" % (1 - test_symbols/total_symbols, test_symbols/total_symbols))
        
        
        for output in ['train.txt', 'test.txt']:
            output_file = os.path.join(PATH, ("en_" if name == 'alice_in_wonderland.txt' else 'be_') + output)
            with open(output_file, 'w') as out:
                out.write("\n".join(train_set) if output == 'train.txt' else "\n".join(test_set))

train-test-split: 0.799879 0.200121
train-test-split: 0.799472 0.200528
